# Define model

In [131]:
from sets import Set 
config = {'models': [
    {'name': 'Person', 'fields': [
            {'name': 'first_name', 'type': 'char', 'length': 30},
            {'name': 'last_name', 'type': 'char', 'length': 30},
            {'name': 'dob', 'type': 'date'},
            {'name': 'rating', 'type': 'int'}
        ],
     'generate_rest_view': True
    },
    {'name': 'Test', 'fields': [
            {'name': 'first_name', 'type': 'char', 'length': 30},
            {'name': 'last_name', 'type': 'char', 'length': 30},
            {'name': 'dob', 'type': 'date'},
            {'name': 'rating', 'type': 'int'}
        ]
    }
],
          'template': 'cosmos',
         'project_name': 'djangolanggen'}

# File Writer

In [135]:
import os

class FileWriter():
    
    def generate_files(self, files, directory = '.'):
        write_path = directory + '/' + files['name']
        if files['type'] == 'dir':
            if not os.path.exists(write_path):
                os.makedirs(write_path)
            for f in files['files']:
                self.generate_files(f, write_path)
        else:
            with open(write_path, 'w') as f:
                f.write(files['content'])
                f.close()

# Generators

In [ ]:
class Generator():
    def generate_files(self, config):
        return None


class DjangoGenerator(Generator):
    
    def chargen(self, field):
        return "{0} = models.CharField(max_length={1})".format(
        field['name'], str(field['length'])
        )

    def dategen(self, field):
        return "{0} = models.DateField()".format(
        field['name']
        )
    
    def individual_field(self, i):
        gen_field_map = {
            'char': chargen,
            'date': dategen,
            'int': intgen
        }
        return gen_field_map[i['type']](i)
    
    def find_types(self, models):
        types = Set()
        for model in models:
            for f in model['fields']:
                types.add(f['type'])
        return types
    
    def import_include(self, models):
        imports = Set()

        def_import = {
            'char': 'from django.db import models',
            'date': 'from django.db import models',
            'int': 'from django.db import models'
        }
        types = self.find_types(models)
        for t in types:
            imports.add(def_import[t])
        return imports

    def intgen(self, field):
        return "{0} = models.IntegerField()".format(
        field['name']
        )
    
    def field_gen(self, fields):
        field_str = []
        for i in fields:
            field_str.append(self.individual_field(i))
        return field_str
     
    def individual_class(self, i):
        str = ''
        str += 'class {0}(models.Model):\n'.format(i['name'])
        for f in self.field_gen(i['fields']):
            str += '    '+f+'\n'

        str += '\n\n'
        return str
    
    def generate_model_rest_view(self, model):
        return ''
    
    def add_template_statics(self, template):
        return {
            'type': 'dir',
            'name': 'templates',
            'files': []
        }
        
    
    def generate_files(self, config):
        str_class = ''
        files = {}
        for i in self.import_include(config['models']):
            str_class += i
        str_class += '\n\n\n'
        for i in config['models']:
            str_class += self.individual_class(i)
    
        files = {'type': 'dir', 'name': config['project_name'], 'files': [
                self.add_template_statics(config['template'])
            ]}
                                                                         
        files['files'].append({'name': 'models.py', 'content': str_class, 'type': 'file'})
        
        model_views = ''
        
        for i in config['models']:
            if 'generate_rest_view' in i and i['generate_rest_view']:
                model_views += self.generate_model_rest_view(i)
                
        if model_views != '':
            files['model_views.py'] = model_views
        return files


class GeneratorFactory():
    
    def return_factory(self, type_to_generate):
        if type_to_generate == 'Django':
            return DjangoGenerator   
    

def classgen(models):
    generator_factory = GeneratorFactory()
    generator = generator_factory.return_factory('Django')()
    file_writer = FileWriter()
    file_writer.generate_files(generator.generate_files(config))
    

classgen(config)